# Imports and Installs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Selenium

In [ ]:
# Set up for running selenium in Google Colab
%%shell
sudo apt -y update
sudo apt-get update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,078 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,223 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,002 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:13 https:/

In [ ]:
! pip install chromedriver-autoinstaller

In [ ]:
import chromedriver_autoinstaller

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

## Other

In [ ]:
import os.path
import pandas as pd
import pickle
import sys
import time
import urllib.request

from bs4 import BeautifulSoup

# Global Variables

In [ ]:
project_dir = "/content/drive/MyDrive/Classes/CSCE 5810 BioComputing/5810 Group Project/"
data_dir = project_dir + "Data/"

# Utillity Functions

In [ ]:
# Get page html
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/
def get_soup(url):
    fp = urllib.request.urlopen(url)
    content = fp.read().decode("utf8")
    fp.close()
    return BeautifulSoup(content, "html.parser")

In [ ]:
# Save to csv
def save_df_to_csv(f_name, df):
    if not os.path.exists(f_name):
        df.to_csv(f_name, index=False)
    else:
        print("File already exists.")

# Scrapers

## The MIT/ICBP siRNA Database

http://web.mit.edu/sirna/index.html

In [ ]:
# Get data from a page
def get_page_data(url):
    p_type = False
    o_type = False
    data = {}

    # Get page content
    soup = get_soup(url)

    # Search for table
    search = soup.find_all("td", align="left", valign="top")
    for s in search:
        vals = [x.strip() for x in s.text.split("\n")]

        # Check for siRNA
        if vals[0] == "Probe type:" and vals[1] == "siRNA":
            p_type = True

        # Check for human data
        elif vals[0] == "Organism:" and vals[1] == "Human":
            o_type = True

        # Get sequences
        elif vals[0] == "Sense sequence:" or vals[0] == "Anti-sense sequence:":
            k = vals[0].replace(":", "")
            k = k.replace(" ", "_")
            k = k.lower()
            data.update({k: vals[1]})

        # Get NCBI URL
        elif vals[0] == "Target gene:":
            data.update({"gene_name": vals[1].split(" ")[0]})
            data.update({"ncbi_url": s.a["href"]})

    # Return data if checks pass
    if p_type and o_type:
        return data
    else:
        return None

In [ ]:
# Scrape URLs from table page

# Get page html
url = "http://web.mit.edu/sirna/sirnas-human.html"
soup = get_soup(url)

# Iterate over table
table = soup.find_all("table", width="400")[0]
url_prefix = "http://web.mit.edu/sirna/"
mit_data = []
for row in table:

    # Get page data
    try:
        p_data = get_page_data("http://web.mit.edu/sirna/" + row.find_all("a")[0]["href"])
        if p_data is not None:
            mit_data.append(p_data)

            # if len(mit_data) == 5:
            #     break

    except:
        continue

In [ ]:
# Convert to dataframe

# Get column names
cols = []
for k in mit_data[0]:
    cols.append(k)

# Convert dictionary to matrix
rows = []
for r in mit_data:
    new = []
    for k in cols:
        if k not in r:
            continue
        new.append(r[k])
    rows.append(new)

# Pandas
mit_d_df = pd.DataFrame(rows, columns=cols)
# mit_df.head()

In [ ]:
# Save to csv
save_df_to_csv(data_dir + "mit_data.csv", mit_d_df)

File already exists.


## Kaggle: siRNA Activity

https://www.kaggle.com/datasets/livtoft/sirna-activity

In [ ]:
kaggle_d_df = pd.read_csv(data_dir + "kaggle_data.csv")
kaggle_d_df = kaggle_d_df[["Target seq", "Start", "End", "Sequence"]]
kaggle_d_df.head()

,Target seq,Start,End,Sequence
0,M60857,195,213,AUUAUCCACUGUUUUUGGA
1,M60857,197,215,AAAUUAUCCACUGUUUUUG
2,M60857,199,217,CAAAAUUAUCCACUGUUUU
3,M60857,201,219,CACAAAAUUAUCCACUGUU
4,M60857,203,221,GCCACAAAAUUAUCCACUG


## Design of a genome-wide siRNA library using an artificial neural network

https://www.nature.com/articles/nbt1118#Sec13

In [ ]:
with open(data_dir + "Design of a genome-wide siRNA library using an artificial neural network - Supplementary Table 3.txt", "r") as file:
    content = file.readlines()
    print(content[0])

16802 CUAAUAUGUUAAUUGAUUUat X X X 0.462



## Specific residues at every third position of siRNA shape its efficient RNAi activity

https://academic.oup.com/nar/article/35/4/e27/1079934

In [ ]:
df = pd.read_excel(data_dir + "gkl1120_Supplementary_Data/nar-02060-met-g-2006-File012.xls",
                   header=1, index_col=0)
df.head()

,Sequence of the passenger strand,Energy of p.s. 5' end,Energy of g.s. 5' end,g.s.-p.s.(kcal/mol),Activity,Our prediction
position#,,,,,,
1,AUGGUGAGCAAGGGCGAGG,-9.91,-11.55,-1.64,54.748296,37.873
2,UGGUGAGCAAGGGCGAGGA,-11.02,-11.14,-0.12,77.713656,52.356
3,GGUGAGCAAGGGCGAGGAG,-12.36,-11.27,1.09,60.159769,47.118
4,GUGAGCAAGGGCGAGGAGC,-11.18,-13.31,-2.13,38.566214,42.939
5,UGAGCAAGGGCGAGGAGCU,-11.26,-10.93,0.33,65.082295,61.027


## NCBI

https://nariyoo.com/python-how-to-run-selenium-in-google-colab/

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

WebDriverException: ignored

In [ ]:
# Set driver path
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

# Setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Install driver from path
chromedriver_autoinstaller.install()

# Initialize the webdriver
driver = webdriver.Chrome(options=chrome_options)

SessionNotCreatedException: ignored

In [ ]:
# Get FASTA url
def get_fasta_url(soup):
    for s in soup.find_all("a", href=True):
        href = s["href"]
        if "report=fasta" in href:
            return "https://www.ncbi.nlm.nih.gov" + href

In [ ]:
# Handle FASTA pages
def get_seq_from_fasta_page(soup):

    # Get text
    text = soup.get_text()

    # Split text
    if ">" in text:
        text = text.split(">")[1]
        text = text.split("\n")[1:]

        # Collect sequence
        seq = ""
        for r in text:
            r = r.strip()
            if r == "":
                break
            seq = seq + r

        return seq
    return None

### MIT/ICBP Links

In [ ]:
# Initilize the dictionary
mit_seqs = {}

In [ ]:
# NOTE: Run this cell repeatedly to get all sequences --------------------------

# Iterate over data
for i, row in enumerate(mit_data):
    name = row["gene_name"]
    if name not in mit_seqs:

        # Get page
        soup = get_soup(row["ncbi_url"])

        # Get FASTA page
        driver.get(get_fasta_url(soup))
        soup = BeautifulSoup(driver.page_source)

        # Get sequence from page
        seq = get_seq_from_fasta_page(soup)

        # Save to dictionary
        if name in mit_seqs:
            print("ERROR:", name, "is already in the dictionary.")
        elif seq is not None:
            mit_seqs.update({name: seq})

        # if i == 0:
        #     break

In [ ]:
# Get names of target genes
names = set()
for i, row in enumerate(mit_data):
    n = row["gene_name"]
    names.add(n)

# Count number of missing sequences
c = 0
for n in list(names):
    if n not in mit_seqs:
        c += 1

        # Display
        print("Missing", n)
print("\nMissing %d/%d, or %f"%(c, len(list(names)), c/len(list(names))))


Missing 0/65, or 0.000000


In [ ]:
# Convert to dataframe
rows = []
for k in mit_seqs:
    rows.append([k, mit_seqs[k]])
mit_s_df = pd.DataFrame(rows, columns=["target_gene", "target_gene_sequence"])
mit_s_df.head()

,target_gene,target_gene_sequence
0,AKT3,GCAGCCCTTCGCTTGCCCTCCCGCCGGCCGCACCGGGCTCCAGGAG...
1,BRMS1,AAGCACCGATAGGCTCTGCCTCCCGAAGAAAAGGGAGCCGCGCAGC...
2,CTNNB1,AAGCCTCTCGGTCTGTGGCAGCAGCGTTGGCCCGGCCCCGGGAGCG...
3,CUL2,CTCGGCGCACAGTAATGGCAGCGCTGTGAGCAGGGGAACGCGAGCT...
4,EGFR,AGACGTCCGGGCAGCCCCCGGCGCAGCGCGGCCGCAGCAGCCTCCG...


In [ ]:
# Save to csv
save_df_to_csv(data_dir + "mit_seqs.csv", mit_s_df)

### Kaggle Target Sequence

In [ ]:
# Iterate over target sequences
kaggle_s_df = []
for k in kaggle_d_df["Target seq"].unique():
    f_name = data_dir + "Kaggle NCBI Sequences/" + k + ".txt"
    if os.path.exists(f_name) and k != "LaminA":

        # Read .txt files
        with open(f_name, "r") as file:
            content = file.readlines()

            # Build NCBI sequences dictionary
            seq = "".join([line.strip() for line in content])
            kaggle_s_df.append([k, seq])

In [ ]:
# Build datafarme
kaggle_s_df = pd.DataFrame(kaggle_s_df, columns=["target_gene", "target_gene_sequence"])
kaggle_s_df.head()

,target_gene,target_gene_sequence
0,M60857,CGGGAACGCAACATGAAGGTGCTCCTTGCCGCCGCCCTCATCGCGG...
1,U47298,GGTACCGAGCTCTTACGCGTGCTAGCCCGGGCTCGAGATCTGCGAT...
2,J03132,GCGCCCCAGTCGACGCTGAGCTCCTCTGCTACTCAGAGTTGCAACC...
3,U92436,CCTCCCCTCGCCCGGCGCGGTCCCGTCCGCCTCTCGCTCGCCTCCC...
4,NM_002037,AGAGCATCAGCAAGAGTAGCAGCGAGCAGCCGCGCTGGTGGCGGCG...


In [ ]:
# Save to csv
save_df_to_csv(data_dir + "kaggle_seqs.csv", kaggle_s_df)

### Cleanup

In [ ]:
driver.quit()

# Data Assembly

## MIT/ICBP

In [ ]:
# Open files (not necessary to run scraper code)
mit_d_df = pd.read_csv(data_dir + "mit_data.csv")
mit_s_df = pd.read_csv(data_dir + "mit_seqs.csv")

In [ ]:
# Convert seqs to dictionary for later convienence
mit_s = {}
for i, row in mit_s_df.iterrows():
    mit_s.update({row["target_gene"]: row["target_gene_sequence"]})

In [ ]:
# Create combined dataframe
mit_df = []
for i, row in mit_d_df.iterrows():
    new = []
    new.append(row["gene_name"])
    new.append(mit_s[row["gene_name"]])
    new.append(row["sense_sequence"])
    new.append(row["anti-sense_sequence"])
    mit_df.append(new)
mit_df = pd.DataFrame(mit_df, columns=["gene_name", "gene", "sense_sequence", "anti-sense_sequence"])
mit_df

,gene_name,gene,sense_sequence,anti-sense_sequence
0,ABCB1,TATTCAGATATTCTCCAGATTCCTAAAGATTAGAGATCATTTCTCA...,CGAACACATTGGAAGGAAATT,TTTCCTTCCAATGTGTTCGGC
1,ABCC1,GCCGCCGCCGCCGCCGCCAGCGCTAGCGCCAGCAGCCGGGCCCGAT...,GAGTGGAATTCCGGAACTATT,TAGTTCCGGAATTCCACTCGG
2,ADAM8,AGACCGTGTCCCGCGGGGCTGTCACCTCCGCCTCTGCTCCCCGACC...,GCATCATCGTCTACCGCAATT,TTGCGGTAGACGATGATGCCT
3,AKT2,GCCGTGCTAGCCGTTGGGCCTGCCTCGGAGGAGGCGTCGCCGCCGC...,GGATGAAGTCGCTCACACATT,TGTGTGAGCGACTTCATCCTT
4,AKT3,GCAGCCCTTCGCTTGCCCTCCCGCCGGCCGCACCGGGCTCCAGGAG...,GCAGCTCCAACTTATATAATT,TTATATAAGTTGGAGCTGCGT
...,...,...,...,...
89,TOP1,CAAATGCGAACTTAGGCTGTTACACAACTGCTGGGGTCTGTTCTCG...,GGATGATGCTGATTATAAATT,TTTATAATCAGCATCATCCTC
90,VIL2,GAGTGTCGGGCGCGGCAGGAGGACGAGGCAGGGCGGGCGGGCGCTC...,GGACTGATTGAATTACGGATT,TCCGTAATTCAATCAGTCCTG
91,Xrn1,CGGCGGAGCCCCGGCGGGGCGTTTGGTTTCGGTTTGGCCCTGACTG...,NaN,NaN
92,Xrn1,CGGCGGAGCCCCGGCGGGGCGTTTGGTTTCGGTTTGGCCCTGACTG...,NaN,NaN


In [ ]:
# Save to csv
save_df_to_csv(data_dir + "mit.csv", mit_df)

## Kaggle

In [ ]:
# Open files (not necessary to run scraper code)
kaggle_d_df = pd.read_csv(data_dir + "kaggle_data.csv")
kaggle_s_df = pd.read_csv(data_dir + "kaggle_seqs.csv")

In [ ]:
# Convert seqs to dictionary for later convienence
kaggle_s = {}
for i, row in kaggle_s_df.iterrows():
    kaggle_s.update({row["target_gene"]: row["target_gene_sequence"]})

In [ ]:
# Create combined dataframe
cols = list(kaggle_d_df.columns)
kaggle_df = []
for i, row in kaggle_d_df.iterrows():
    if row["Target seq"] in kaggle_s:
        new = []
        for c in cols:
            new.append(row[c])
        new.append(kaggle_s[row["Target seq"]])
        kaggle_df.append(new)
cols.append("Target gene")
kaggle_df = pd.DataFrame(kaggle_df, columns=cols)
kaggle_df

,Target seq,Start,End,Sequence,G,U,bi,uni,duplex,Pos1,...,Dif_5-3,Content+,Content-,Cons+,Cons-,Cons_Sum,Hyb19,target,Activity,Target gene
0,M60857,195,213,AUUAUCCACUGUUUUUGGA,3,9,-7.0,-1.9,-28.1,-1.1,...,-1.3,2,6,2,6,4,0.001175,-8.1,24.7,CGGGAACGCAACATGAAGGTGCTCCTTGCCGCCGCCCTCATCGCGG...
1,M60857,197,215,AAAUUAUCCACUGUUUUUG,2,9,-0.7,0.0,-24.2,-0.9,...,-1.2,1,6,1,5,4,0.001175,-8.1,9.8,CGGGAACGCAACATGAAGGTGCTCCTTGCCGCCGCCCTCATCGCGG...
2,M60857,199,217,CAAAAUUAUCCACUGUUUU,1,8,-1.5,0.0,-24.2,-2.1,...,1.2,2,5,3,2,-1,0.001175,-11.4,50.5,CGGGAACGCAACATGAAGGTGCTCCTTGCCGCCGCCCTCATCGCGG...
3,M60857,201,219,CACAAAAUUAUCCACUGUU,1,6,-0.6,0.0,-26.7,-2.1,...,1.2,3,3,3,3,0,0.001175,-14.5,83.7,CGGGAACGCAACATGAAGGTGCTCCTTGCCGCCGCCCTCATCGCGG...
4,M60857,203,221,GCCACAAAAUUAUCCACUG,2,4,-0.1,0.0,-30.3,-3.4,...,1.3,4,2,2,3,1,0.001175,-17.8,60.3,CGGGAACGCAACATGAAGGTGCTCCTTGCCGCCGCCCTCATCGCGG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,M33197,717,735,GUUCAGCUCAGGGAUGACC,6,4,-9.0,-0.3,-37.8,-2.2,...,-1.1,5,3,8,6,-2,0.000789,-15.1,26.0,GTTCGACAGTCAGCCGCATCTTCTTTTGCGTCGCCAGCCGAGCCAC...
605,M33197,837,855,CGCCUGCUUCACCACCUUC,2,5,-2.8,0.0,-38.6,-2.4,...,0.0,4,4,5,3,-2,0.000789,-24.8,48.0,GTTCGACAGTCAGCCGCATCTTCTTTTGCGTCGCCAGCCGAGCCAC...
606,M33197,972,990,GAGCUUGACAAAGUGGUCG,7,4,-7.9,-2.3,-34.1,-2.4,...,0.0,3,2,5,4,-1,0.000789,-5.2,36.0,GTTCGACAGTCAGCCGCATCTTCTTTTGCGTCGCCAGCCGAGCCAC...
607,M33197,1008,1026,GUUGCUGUAGCCAAAUUCG,5,6,-9.5,-0.3,-31.7,-2.2,...,-0.2,3,3,2,4,2,0.000789,-11.6,25.0,GTTCGACAGTCAGCCGCATCTTCTTTTGCGTCGCCAGCCGAGCCAC...


In [ ]:
# Save to csv
save_df_to_csv(data_dir + "kaggle.csv", kaggle_df)